# Import packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
from copy import deepcopy
import pickle
import pandas as pd
import numpy as np

# Load business data

In [2]:
df = pd.read_pickle('/Users/mai/Desktop/yelp_dataset/to_submit/businessdf.pickle')
df.head()

,business_id,categories,city,business_status,latitude,longitude,business_name,review_count_per_business,average_stars_per_business,US_state,...,meal_brunch,meal_dessert,meal_dinner,meal_latenight,meal_lunch,parking_garage,parking_lot,parking_street,parking_valet,parking_validated
0,QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,1,43.605499,-79.652289,Emerald Chinese Restaurant,128,2.5,ON,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,1,35.092564,-80.859132,Musashi Japanese Restaurant,170,4.0,NC,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,3,5.0,AZ,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,1,35.190012,-80.887223,Queen City Plumbing,4,4.0,NC,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,68dUKd8_8liJ7in4aWOSEA,"Shipping Centers, Couriers & Delivery Services...",Mississauga,1,43.599475,-79.711584,The UPS Store,3,2.5,ON,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Extract Restaurant or food store information

In [3]:
# Specify rows data that contains 'Restaurants' or 'Food' in the 
restaurants = df[df['categories'].str.contains('Restaurant|Food') == True]

In [4]:
# Check the shape
restaurants.shape

(74599, 105)

In [8]:
# Check if I extracted restaurant and food data successfully 
restaurants.categories[:5]

0     Specialty Food, Restaurants, Dim Sum, Imported...
1                     Sushi Bars, Restaurants, Japanese
10    Restaurants, Breakfast & Brunch, Mexican, Taco...
12           Italian, Restaurants, Pizza, Chicken Wings
13                                       Bakeries, Food
Name: categories, dtype: object

In [9]:
# Put labels to business information for future use
Business_type = ['Restaurants', 'Food']
for b in Business_type:
    restaurants.loc[restaurants['categories'].str.contains(b),'business_type'] = b

In [10]:
# Those with NaN values after the above step are 'Restaurant Supplies', thus, I decided to drop the information
restaurants['business_type'].dropna(inplace=True)

# Create a column of couisine information

In [11]:
# Create a cuisine column based on a popular cuisine list
cuisines = ['African', 'Algerian', 'American', 'Belgian', 'Brazilian', 'British', \
            'Cajun', 'Canadian', 'Chinese', 'Cuban', 'Egyptian', 'French', \
            'German', 'Greek', 'Haitian', 'Hawaiian', 'Indian', 'Irish', 'Italian', 
            'Japanese', 'Jewish', 'Kenyan', 'Korean', 'Latvian', \
            'Libyan', 'Mediterranean', 'Mexican', 'Middle Eastern', 'Mormon', \
            'Nigerian', 'Peruvian', 'Polish', 'Portuguese', 'Russian', \
            'Salvadorian', 'Scottish', 'Spanish', 'Swedish', \
            'Tahitian', 'Thai', 'Tibetan', 'Turkish', 'Vietnamese','Welsh'\
            'Venezuelan', 'Argentine', 'Cantonese', 'Salvadoran', 'Arabian',\
            'Moroccan', 'Russian', 'Uzbek', 'Armenian', 'Lebanese', 'Indonesian', \
            'Singaporean']

In [12]:
# Find a cuisine name in the 'categories' column and create a new colomn (or fill 1 in the relevant column)
# E.g. When a cell contains 'American', it creates an 'American' column or fill 1 in the 'American' column if it exists already
for c in cuisines:
    restaurants.loc[restaurants['categories'].str.contains(c),'cuisine'] = c

# Create a column of food type information

In [13]:
# Create a food type column based on a popular cuisine list
Food_type = ['Arabian', 'Armenian', 'Asian Fusion', 'Barbeque', 'Basque', 
            'Breakfast & Brunch', 'Buffets', 'Burgers', 'Cafes', \
            'Cafeteria', 'Cajun', 'Cheesesteaks', 'Chicken Shops', \
            'Chicken Wings', 'Coffee & Tea', 'Comfort Food', 'Creperies', \
            'Delis', 'Dim Sum', 'Diners', 'Falafel', 'Fast Food', 'Fish & Chips', \
            'Fondue', 'Food Court', 'Food Stands', 'Gastropubs', 'Halal', \
            'Hot Dogs', 'Hot Pot', 'Kebab', 'Kosher', 'Moroccan', 'Noodles',\
            'Pizza', 'Ramen', 'Russian', 'Salad', 'Salvadoran', 'Sandwiches', \
            'Seafood', 'Soup', 'Steakhouses', 'Sushi Bars', 'Tacos', 'Tapas Bars', \
            'Tapas Bars', 'Uzbek', 'Waffles', 'Wraps', ]

In [14]:
# Do the same as 'cuisine' columns
for f in Food_type:
    restaurants.loc[restaurants['categories'].str.contains(f),'food_type'] = f

# Dummify columns 

In [15]:
dummy_business_type = pd.get_dummies(restaurants[['business_type']], drop_first=True)

In [16]:
dummy_cuisine_and_food = pd.get_dummies(restaurants[['cuisine', 'food_type']])

In [17]:
restaurants_ver2 = pd.concat([restaurants, dummy_business_type, dummy_cuisine_and_food], axis=1)

In [18]:
restaurants_ver2 = restaurants_ver2.drop(['business_type', 'categories', 'cuisine', 'food_type'], axis=1)

# Feature generation 

## Create a column for total checkin counts

In [19]:
# Load check-in data
checkins = pd.read_pickle('/Users/mai/Desktop/yelp_dataset/to_submit/checkindf.pickle')
checkins.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [ ]:
# Create a new DataFrame to hold the timeseries of the checkins
df_checkins = pd.DataFrame()

for id in range(len(checkins.date)):
    
    df_checkins_id = pd.DataFrame()
    
    split = checkins.date[id].split(',')
    df_checkins_id['Checkin_date'] = pd.to_datetime(split)
    df_checkins_id['Business_id'] = checkins.business_id[id]
    df_checkins_id['Year'] = df_checkins_id['checkin_date'].dt.year
    

    df_checkins_id = df_checkins_id.groupby(['business_id','year']).agg(['count']).reset_index()
#    print(df_checkins_id)

    df_checkins = pd.concat([df_checkins, df_checkins_id])

In [ ]:
# Pickle results to avoid running all the above code again - it took hours to finish running
df_checkins.to_pickle('df_checkins.pickle')

In [6]:
# Load data
df_checkins = pd.read_pickle('/Users/mai/Desktop/yelp_dataset/to_submit/df_checkins.pickle')
df_checkins.head()

,business_id,year,number_checkins
0,--1UhMGODdWsrMastO9DZw,2016,4
1,--1UhMGODdWsrMastO9DZw,2017,2
2,--6MefnULPED_I942VcFNA,2011,2
3,--6MefnULPED_I942VcFNA,2012,5
4,--6MefnULPED_I942VcFNA,2013,20


In [7]:
# Check the DataFrame shape
df_checkins.shape

(713088, 3)

In [23]:
# Rename the column to make it more clear
df_checkins.rename(columns={'number_checkins': 'checkins_peryear'}, inplace=True)

In [24]:
# Current DataFrame structure: one check-in information per row
# Group the checkin information by business_id
total_checkins = pd.DataFrame(df_checkins.groupby('business_id')['checkins_peryear'].sum()).reset_index()

In [25]:
# Estimate the number of years the business is listed on Yelp
total_checkins_yearcount = pd.DataFrame(df_checkins.groupby('business_id')['year'].count()).reset_index()

In [26]:
# Put all data in a DataFrame
df_checkins_ver2 = total_checkins.merge(total_checkins_yearcount, on='business_id')
df_checkins_ver2.columns=['business_id', 'total_checkins', 'total_checkinyear_count']
df_checkins_ver2.head()

,business_id,total_checkins,total_checkinyear_count
0,--1UhMGODdWsrMastO9DZw,6,2
1,--6MefnULPED_I942VcFNA,167,8
2,--7zmmkVg-IMGaXbuVd0SQ,180,5
3,--8LPVSo5i0Oo61X01sV9A,1,1
4,--9QQLMTbFzLJ_oT-ON3Xw,37,9


In [27]:
# Put all data in a DataFrame
restaurants_ver3 = df_checkins_ver2.merge(restaurants_ver2, on='business_id')

## Create a column for estimated operating years

In [28]:
# Create a copy to avoid overwrting
df_operatingyears = checkins.copy()

In [29]:
# Check how the DataFrame looks like
df_operatingyears[['business_id', 'date']].head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [30]:
# Find the first review year

minyear = []

def minyear_extract(x):
    data = x.split(',')
    data = [int(x.strip()[:4]) for x in data]
    minyear.append(min(data))
    return min(data)

In [31]:
# Create a column that contains first review year information
df_operatingyears['minyear'] = df_operatingyears.date.apply(minyear_extract)

In [32]:
# Find the latest review year

maxyear = []

def maxyear_extract(x):
    data = x.split(',')
    data = [int(x.strip()[:4]) for x in data]
    maxyear.append(max(data))
    return max(data)

In [33]:
# Create a column that contains latest review year information
df_operatingyears['maxyear'] = df_operatingyears.date.apply(maxyear_extract)

In [34]:
# Estimate the operating years 
df_operatingyears['estimated_opearating_years'] = df_operatingyears['maxyear'] - df_operatingyears['minyear']

In [ ]:
# Put all data in a DataFrame
restaurants_ver4 = df_operatingyears.merge(p_restaurants_ver3, on='business_id')

In [ ]:
# Drop the date column since I don't need to use it again
restaurants_ver4.drop(columns='date', inplace=True)

## Create a column for total review count

In [8]:
# Load review data
reviewdf = pd.read_pickle('/Users/mai/Desktop/yelp_dataset/to_submit/reviewdf.pickle')
reviewdf.head(2)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1.0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6.0,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0.0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0.0,yXQM5uF2jS6es16SJzNHfg


In [9]:
# Group reviews by business_id
total_reviews = pd.DataFrame(reviewdf.groupby('business_id').count()['review_id'])
total_reviews = total_reviews.reset_index()

In [ ]:
# Put all data in a DataFrame
restaurants_ver5 = total_reviews.merge(restaurants_ver4, on='business_id')

# Create predictor variables

In [ ]:
# Create categories per business type
# Businesses that is active and run for 3+ years: Successful
# Businesses that is active but run for less than 3 years: Active for less than three years
# Busienss that is closed: Failed
predictions = []

for o in range(len(restaurants_ver5)):
    if (restaurants_ver5['estimated_opearating_years'][o] >= 3) & (restaurants_ver5['business_status'][o] == 1):
        predictions.append('successful')
    elif (restaurants_ver5['estimated_opearating_years'][o] < 3) & (restaurants_ver5['business_status'][o] == 1):
        predictions.append('active for less than three years')
    else:
        predictions.append('failed')

In [ ]:
# Create a new column
restaurants_ver5['prediction_col'] = predictions

In [ ]:
# Check the distribution
total = len(restaurants_ver5['prediction_col'])
successful_count = len(restaurants_ver5[restaurants_ver5['prediction_col'] == 'successful'])
active_count = len(restaurants_ver5[restaurants_ver5['prediction_col'] == 'active for less than three years'])
failed_count = len(restaurants_ver5[restaurants_ver5['prediction_col'] == 'failed'])

In [ ]:
# Remove data that are not necessary for this capstone
# This capstone predicts 'successful' or 'failed', thus other information need to be removed
restaurants_clean = restaurants_ver5[restaurants_ver5['prediction_col'] != 'active for less than three years']

In [ ]:
# Pickle results to avoid running all cells 
restaurants_clean.to_pickle('restaurants_clean.pickle')

In [46]:
# Load data
p_restaurants_clean = pd.read_pickle('/Users/mai/Desktop/yelp_dataset/to_submit/restaurants_clean.pickle')
p_restaurants_clean.head()

,business_id,review_id,minyear,maxyear,estimated_opearating_years,total_checkins,total_checkinyear_count,city,business_status,latitude,...,food_type_Seafood,food_type_Soup,food_type_Steakhouses,food_type_Sushi Bars,food_type_Tacos,food_type_Tapas Bars,food_type_Uzbek,food_type_Waffles,food_type_Wraps,predictor_col
1,--6MefnULPED_I942VcFNA,46,2011,2018,7,167,8,Richmond Hill,1,43.841694,...,0,0,0,0,0,0,0,0,0,successful
2,--7zmmkVg-IMGaXbuVd0SQ,59,2014,2018,4,180,5,Huntersville,1,35.437106,...,0,0,0,0,0,0,0,0,0,successful
3,--9e1ONYQuAa-CB_Rrw7Tw,1661,2010,2018,8,2770,9,Las Vegas,1,36.123183,...,0,0,1,0,0,0,0,0,0,successful
4,--DaPTJW3-tB1vP-PfdTEg,49,2012,2018,6,95,7,Toronto,1,43.677807,...,0,0,0,0,0,0,0,0,0,successful
5,--FBCX-N37CMYDfs790Bnw,142,2010,2018,8,528,9,Henderson,1,35.978679,...,0,0,0,0,0,0,0,0,0,successful
